# =============================================
## GHI CHÚ TỔNG QUAN (NOTE – DÀNH CHO KAGGLE)
# =============================================
 **Mục tiêu**
 - Fine-tune mô hình T5 (seq2seq) cho bài toán Text-to-SQL trên Kaggle.
 - Dùng dataset công khai (b-mc2/sql-create-context, Clinton/Text-to-sql-v1, knowrohit07/know_sql),
 sau đó trộn, tiền xử lý (tokenize), huấn luyện, lưu mô hình.
 - Không thay đổi logic cốt lõi; chỉ bổ sung chú thích chi tiết.

 **Luồng chính**
 1) Cài đặt thư viện → Import.
 2) Nạp/ghép dataset có sẵn → lưu cache (merged_dataset).
 3) Tiền xử lý (tokenize) → lưu cache (tokenized_datasets).
 4) Kiểm thử zero-shot (baseline) trên model gốc.
 5) Huấn luyện full fine-tune → lưu mô hình đã fine-tune.
 6) Kiểm thử lại với mô hình fine-tuned + đánh giá (ROUGE) mẫu nhỏ.
 7) Đóng gói mô hình (zip) để tải xuống / dùng về sau.

 **Đầu ra (sản phẩm)**
 - Thư mục model đã fine-tune: /kaggle/working/sql_t5_finetuned
 - Có thể nạp về app (FastAPI/Flask/.NET-bridge) bằng from_pretrained("/kaggle/working/sql_t5_finetuned").

In [ ]:
# Cài đặt thư viện datasets nếu chưa có
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.

In [ ]:
# Cài đặt thư viện evaluate
!pip install evaluate 


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


# ==============================
# 2) IMPORT CÁC THƯ VIỆN CẦN THIẾT
# ==============================

In [ ]:
import torch; 
print(torch.__version__, torch.cuda.is_available())

2.6.0+cu124 True


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict, load_dataset, interleave_datasets, load_from_disk # Dataset Huggingface
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer # Model và Tokenizer cho trainning
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor
import time # Ghi thời gian trainning
import evaluate # Thư viện evaluate để đánh giá mô hình (BLEU, ROUGE)
import pandas as pd # Thư viện pandas để xử lý dữ liệu dạng bảng/kết quả
import numpy as np # Thư viện numpy để xử lý mảng


import warnings
warnings.filterwarnings("ignore") # Bỏ qua các cảnh báo không cần thiết

2025-10-16 10:29:36.576791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760610576.808863      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760610576.881376      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Import thư viện transformers và huggingface_hub
!pip install transformers huggingface_hub 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 10.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 1.0.0rc2
    Uninstalling huggingface-hub-1.0.0rc2:
      Successfully uninstalled huggingface-hub-1.0.0rc2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic 2.12.0a1 which is incompatible.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration # Model và Tokenizer T5

import os # Thư viện os để thao tác với hệ thống tệp

# ==============================
# 3) CẤU HÌNH ĐƯỜNG DẪN VÀ THIẾT BỊ
# ==============================

In [ ]:
drive_model_path = '/kaggle/working/' # Đường dẫn lưu model trên Kaggle

In [ ]:
torch.cuda.is_available() # Kiểm tra GPU có sẵn không, trả về True hoặc False

True

In [ ]:
from huggingface_hub import snapshot_download

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Tải snapshot repo model về LOCAL FOLDER (chỉ gọi API 1 lần, không đụng 'additional_chat_templates')
#   - repo_id 't5-small' sẽ redirect về 'google-t5/t5-small'
local_dir = snapshot_download(repo_id="t5-small") 

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/8.47k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/537 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

...

rust_model.ot:   0%|          | 0.00/242M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

In [ ]:
model_name='t5-small' # Tên model nền; code gốc chọn t5-small cho baseline/finetune

# tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(local_dir, local_files_only=True)

# Trên Kaggle GPU có thể dùng bfloat16, còn local CPU thì để mặc định FP32
if device == "cuda":
    original_model = T5ForConditionalGeneration.from_pretrained(
        model_name, torch_dtype=torch.bfloat16
    ).to(device)
else:
    original_model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)

import warnings
warnings.filterwarnings("ignore", message="You are using the default legacy behaviour")


# ==============================
# 4) NẠP/CHUẨN BỊ DATASET (CÓ CACHE)
# ==============================

In [ ]:
# Cố gắng nạp dataset đã merge sẵn từ cache "merged_dataset" để tiết kiệm thời gian
try:
    dataset = load_from_disk("merged_dataset")
    print("Loaded Merged Dataset")
except:
    # Nếu không có cache, tiến hành tải và merge 3 dataset công khai và chia split thủ công
    
    # 4.1) b-mc2/sql-create-context (chứa question/context/answer)
    dataset_scc_train = load_dataset("b-mc2/sql-create-context", split='train[:80%]') # 80% train
    dataset_scc_test  = load_dataset("b-mc2/sql-create-context", split='train[-20%:-10%]') # 10% test
    dataset_scc_val   = load_dataset("b-mc2/sql-create-context", split='train[-10%:]') # 10% validation

    # 4.2) Clinton/Text-to-sql-v1 (đổi tên cột để thống nhất question/context/answer)

    dataset_tts_train = load_dataset("Clinton/Text-to-sql-v1", split='train[:80%]') # 80% train
    dataset_tts_train = dataset_tts_train.remove_columns(['source', 'text']) # Bỏ cột không cần thiết
    dataset_tts_train = dataset_tts_train.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'}) # Đổi tên cột cho thống nhất với dataset_scc
    
    dataset_tts_test  = load_dataset("Clinton/Text-to-sql-v1", split='train[-20%:-10%]') # 10% test
    dataset_tts_test  = dataset_tts_test.remove_columns(['source', 'text']) 
    dataset_tts_test  = dataset_tts_test.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'}) 
    dataset_tts_val   = load_dataset("Clinton/Text-to-sql-v1", split='train[-10%:]') # 10% validation
    dataset_tts_val   = dataset_tts_val.remove_columns(['source', 'text'])
    dataset_tts_val   = dataset_tts_val.rename_columns({'instruction': 'question', 'input': 'context', 'response': 'answer'})

    # 4.3) knowrohit07/know_sql (đổi tên cột để thống nhất question/context/answer)
    dataset_ks_train  = load_dataset("knowrohit07/know_sql", split='validation[:80%]')
    dataset_ks_test   = load_dataset("knowrohit07/know_sql", split='validation[-20%:-10%]')
    dataset_ks_val    = load_dataset("knowrohit07/know_sql", split='validation[-10%:]')

    # 4.4) Ghép 3 nguồn lại bằng interleave_datasets để tăng đa dạng dữ liệu
    dataset = DatasetDict({ 'train': interleave_datasets([dataset_scc_train, dataset_tts_train, dataset_ks_train]),
                            'test': interleave_datasets([dataset_scc_test, dataset_tts_test, dataset_ks_test]),
                            'validation': interleave_datasets([dataset_scc_val, dataset_tts_val, dataset_ks_val])})

    # Lưu dataset đã merge vào cache "merged_dataset" để lần sau dùng lại
    dataset.save_to_disk("merged_dataset")
    print("Merged and Saved Dataset")

dataset # Hiển thị thông tin dataset

README.md: 0.00B [00:00, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/118 [00:00<?, ?B/s]

texttosqlv2.jsonl:   0%|          | 0.00/635M [00:00<?, ?B/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/118695 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14835 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/14838 [00:00<?, ? examples/s]

Merged and Saved Dataset


DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 14838
    })
})

In [ ]:
dataset['test'][0] # Kiểm tra một mẫu trong tập test

{'answer': 'SELECT date FROM table_name_11 WHERE away_team = "essendon"',
 'question': 'On what Date did the Away team essendon play?',
 'context': 'CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)'}

# ==============================
# 5) HÀM TIỀN XỬ LÝ (TOKENIZE) + TẠO PROMPT
# ==============================

Ở bước này, cần chuyển đổi các bộ dữ liệu thành dạng hướng dẫn rõ ràng cho mô hình ngôn ngữ lớn (LLM).

Sau đó, tiến hành tiền xử lý dữ liệu prompt-response bằng cách mã hóa (tokenize) để lấy ra các input_ids phục vụ cho quá trình huấn luyện.

Ghi chú: Chuyển dữ liệu dạng (context/question/answer) → (input_ids/labels) cho T5
 Prompt dạng:
 Tables:\n{context}\n\nQuestion:\n{question}\n\nAnswer:\n

In [ ]:
# def tokenize_function(example): # Hàm tokenize_function để tiền xử lý dữ liệu

# #     print(len(example["question"]))
# # # Tiền tố/giữa/hậu cho prompt để mô hình rõ cấu trúc đầu vào
#     start_prompt = "Tables:\n" # Bắt đầu với Tables:
#     middle_prompt = "\n\nQuestion:\n" # Giữa là Question:
#     end_prompt = "\n\nAnswer:\n" # Kết thúc với Answer:

#     # Ghép promt theo từng cặp context/question
#     data_zip = zip(example['context'], example['question']) # Ghép cặp context và question
#     prompt = [start_prompt + context + middle_prompt + question + end_prompt for context, question in data_zip] # Tạo prompt hoàn chỉnh
    
#     # Mã hoá prompt → input_ids (padding/truncation theo mặc định max_length của tokenizer)
#     example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids # Tokenize prompt
    
#     # Mã hoá câu trả lời (answer) → labels
#     example['labels'] = tokenizer(example['answer'], padding="max_length", truncation=True, return_tensors="pt").input_ids # Tokenize answer
# #     print(prompt[0])
# #     print()

#     return example

# # Ghi chú: Hàm trên sẽ được áp vào cả 3 split (train/validation/test) bằng map(batched=True)
# # batched=True để xử lý theo lô, tăng tốc độ xử lý
# # Hàm tokenize_function xử lý tất cả dữ liệu trên tất cả các split theo lô.


# # Cố gắng nạp dataset đã tokenized sẵn từ cache "tokenized_datasets" để tiết kiệm thời gian
# try:
#     # Thử nạp từ cache nếu đã tokenized trước đó
#     tokenized_datasets = load_from_disk("tokenized_datasets") 
#     print("Loaded Tokenized Dataset")
# except:
#     # Nếu chưa có cache, thực thi map → loại bỏ cột thừa → lưu cache
#     tokenized_datasets = dataset.map(tokenize_function, batched=True) # Áp dụng hàm tokenize_function cho toàn bộ dataset
#     tokenized_datasets = tokenized_datasets.remove_columns(['question', 'context', 'answer']) # Loại bỏ cột thừa để tiết kiệm bộ nhớ

#     # Lưu dataset đã tokenized vào cache "tokenized_datasets" để lần sau dùng lại
#     tokenized_datasets.save_to_disk("tokenized_datasets")
#     print("Tokenized and Saved Dataset")
from transformers import DataCollatorForSeq2Seq

MAX_INPUT_LEN = 256   # độ dài tối đa của prompt (Tables + Question)
MAX_LABEL_LEN = 128   # độ dài tối đa của Answer/SQL

def tokenize_function(example):
    # Tạo prompt chuẩn dạng:
    # Tables: [schema]
    # Question: [natural language question]
    # Answer: [SQL query]
    start_prompt = "Tables:\n"
    middle_prompt = "\n\nQuestion:\n"
    end_prompt = "\n\nAnswer:\n"

    # Ghép từng cặp context + question thành một prompt hoàn chỉnh
    prompt = [
        start_prompt + context + middle_prompt + question + end_prompt
        for context, question in zip(example["context"], example["question"])
    ]

    # Tokenize đầu vào và đầu ra với padding động + cắt độ dài hợp lý
    model_inputs = tokenizer(
        prompt,
        padding=False,  # 🔻 không ép về max_length
        truncation=True,
        max_length=MAX_INPUT_LEN,
    )

    labels = tokenizer(
        example["answer"],
        padding=False,
        truncation=True,
        max_length=MAX_LABEL_LEN,
    )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# =========================================================
# 🔹 TẠO DATASET TOKENIZED (CÓ CACHE)
# =========================================================
try:
    tokenized_datasets = load_from_disk("tokenized_datasets")
    print("✅ Loaded tokenized dataset from cache.")
except:
    tokenized_datasets = dataset.map(
        tokenize_function, batched=True, remove_columns=["question", "context", "answer"]
    )
    tokenized_datasets.save_to_disk("tokenized_datasets")
    print("✅ Tokenized dataset saved to cache.")

# Data collator → tự padding theo batch, giúp tiết kiệm VRAM
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=finetuned_model,
    padding="longest",
)

✅ Loaded tokenized dataset from cache.


In [ ]:
# Hiển thị thông tin tokenized_datasets kiểm tra nhanh khóa/cấu trúc trước khi train
print(tokenized_datasets.keys()) # Hiển thị các split có trong tokenized_datasets
print(tokenized_datasets['train'][0].keys()) # Hiển thị các khóa trong một mẫu của tập train
print(tokenized_datasets['train'][0]['input_ids'][:10]) # Hiển thị 10 token đầu tiên của input_ids
print(tokenized_datasets['train'][0]['labels'][:10]) # Hiển thị 10 token đầu tiên của labels
print(tokenized_datasets) # Hiển thị thông tin dataset

dict_keys(['train', 'test', 'validation'])
dict_keys(['input_ids', 'labels'])
[4398, 7, 10, 205, 4386, 6048, 332, 17098, 819, 41]
[3, 23143, 14196, 2847, 17161, 599, 1935, 61, 21680, 819]
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


In [ ]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}") # In kích thước tập train (số mẫu, số cột)
print(f"Validation: {tokenized_datasets['validation'].shape}") # In kích thước tập validation (số mẫu, số cột)
print(f"Test: {tokenized_datasets['test'].shape}") # In kích thước tập test (số mẫu, số cột)

print(tokenized_datasets) # Hiển thị thông tin tokenized_datasets

Shapes of the datasets:
Training: (118695, 2)
Validation: (14838, 2)
Test: (14835, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 118695
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14835
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 14838
    })
})


# ==============================
# 7) KIỂM THỬ ZERO-SHOT VỚI MODEL GỐC (BASELINE)
# ==============================

In [ ]:
import torch # Thư viện PyTorch tăng tốc độ xử lý GPU, tensor

# Định nghĩa prompt & answer mẫu để test nhanh (người dùng cần thay bằng ví dụ thật)
prompt = "Your input prompt here"  # Ví dụ: Tables + Question (cần thay bằng prompt thực)
answer = "Expected human response here"  # Ví dụ: câu SQL chuẩn tương ứng (đáp án chuẩn)

# Đảm bảo model và input cùng trên một thiết bị (CPU hoặc GPU) nếu có
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Di chuyển model sang thiết bị phù hợp (device = CPU hoặc GPU)
original_model.to(device)

# Tokenize đầu vào input và chuyển nó sang cùng thiết bị với model
inputs = tokenizer(prompt, return_tensors='pt').to(device)

# Sinh đầu ra zero-shot từ model gốc (chưa fine-tune)
output = tokenizer.decode( # dùng tokenizer.decode để giải mã token thành text
    original_model.generate( # dùng phương thức generate để sinh text
        inputs["input_ids"], # input_ids của prompt đã tokenized
        max_new_tokens=200, # Giới hạn tối đa 200 token mới sinh
    )[0], # Lấy mảng token đầu tiên trong batch (ở đây batch size=1)
    skip_special_tokens=True # Bỏ qua các token đặc biệt khi giải mã
)

# In kết quả zero-shot baseline
dash_line = '-' * 100 
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}') 
print(dash_line)
print(f'BASELINE HUMAN ANSWER:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')


----------------------------------------------------------------------------------------------------
INPUT PROMPT:
Your input prompt here
----------------------------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
Expected human response here

----------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Votre prompt here


# ==============================
# 8) HUẤN LUYỆN FULL FINE-TUNE (KAGGLE GPU)
# ==============================

#### 5e-3

Thời gian huấn luyện      = 2 giờ 49 phút 1 giây trên laptop sử dụng notebook Kaggle (PC cá nhân không đủ bộ nhớ CUDA để huấn luyện với tập dữ liệu lớn)

Training Loss   = 0.023100

Validation Loss = 0.013285

---

**Giải thích:**

- **5e-3**: Đây là giá trị learning rate (tốc độ học) được sử dụng khi huấn luyện mô hình. Giá trị này ảnh hưởng đến tốc độ cập nhật trọng số của mô hình trong quá trình học.
- **Thời gian huấn luyện**: Tổng thời gian để hoàn thành quá trình fine-tune mô hình trên GPU của Kaggle. Nếu dùng máy cá nhân (PC) không có đủ bộ nhớ CUDA thì sẽ không thể huấn luyện với tập dữ liệu lớn.
- **Training Loss**: Độ lỗi (loss) trên tập huấn luyện. Giá trị càng nhỏ chứng tỏ mô hình học tốt trên dữ liệu huấn luyện.
- **Validation Loss**: Độ lỗi trên tập kiểm thử (validation). Giá trị này dùng để đánh giá khả năng tổng quát hóa của mô hình trên dữ liệu chưa từng thấy. Nếu validation loss thấp và gần với training loss, mô hình không bị overfit.

**Kết luận:**  
Mô hình đã được huấn luyện với tốc độ học 5e-3, thời gian gần 3 tiếng trên GPU của Kaggle. Kết quả training loss và validation loss đều thấp, chứng tỏ mô hình học tốt và có khả năng tổng quát hóa tốt trên dữ liệu kiểm thử.

In [ ]:
# Nếu trước đó bạn đã dùng snapshot_download(...) và có biến local_dir,
# thì ưu tiên nạp model/tokenizer từ local_dir; nếu không thì dùng tên model gốc.
base_source = local_dir if 'local_dir' in globals() else model_name  # ví dụ: "google-t5/t5-small" hoặc "t5-small"


# ===== Cố gắng nạp model đã fine-tune nếu có =====
try:
    # Nạp lại mô hình đã fine-tune (thư mục local do bạn save_pretrained trước đó)
    finetuned_model = T5ForConditionalGeneration.from_pretrained("finetuned_model_2_epoch").to(device)

    # Nạp tokenizer đi kèm checkpoint fine-tune (đảm bảo đồng bộ vocab)
    tokenizer = T5Tokenizer.from_pretrained("finetuned_model_2_epoch")

    to_train = False  # Có model rồi → không cần train lại

except Exception as e:
    # Không có checkpoint fine-tune → nạp model nền để fine-tune
    to_train = True

    if device == "cuda":
        # Trên Kaggle GPU có thể dùng bfloat16 để tiết kiệm VRAM/tăng tốc
        finetuned_model = T5ForConditionalGeneration.from_pretrained(
            base_source, torch_dtype=torch.bfloat16
        ).to(device)
    else:
        # Trên CPU giữ FP32 mặc định
        finetuned_model = T5ForConditionalGeneration.from_pretrained(base_source).to(device)

    # Tokenizer đồng bộ với nguồn model nền (từ local_dir nếu có, hoặc từ tên model)
    tokenizer = T5Tokenizer.from_pretrained(base_source)

In [ ]:
# import os
# os.environ["WANDB_DISABLED"] = "true" # Tắt Weights & Biases để tránh lỗi khi train trên Kaggle
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"  # giúp giảm phân mảnh VRAM

# if to_train:
#     # Tạo thư mục output duy nhất với timestamp để lưu kết quả train
#     output_dir = f'./sql-training-{str(int(time.time()))}'

#     # Cấu hình tham số huấn luyện
#     training_args = TrainingArguments(
#         output_dir=output_dir, # Thư mục lưu kết quả huấn luyện
#         learning_rate=5e-3, # Tốc độ học của mô hình (càng lớn càng nhanh nhưng dễ quá khớp)
#         num_train_epochs=1, # Số epoch huấn luyện (có thể tăng lên 2,3,... để cải thiện độ chính xác nhưng tốn thời gian hơn)
#         per_device_train_batch_size=8, # Dùng batch size 16 (có thể tăng lên nếu GPU đủ lớn, giúp mô hình học tốt hơn)
#         per_device_eval_batch_size=8, # Dùng batch size 16 cho đánh giá
#         weight_decay=0.01, # Hệ số weight decay để tránh overfitting (thường chọn 0.01)
#         logging_steps=50, # Ghi log mỗi 50 bước
#         eval_strategy="steps", # Đánh giá theo số bước để tránh overfitting, thường chọn 'steps' vì dataset nhỏ
#         eval_steps=500, # Đánh giá mỗi 500 bước
#         fp16=True,                       # bật mixed precision để tiết kiệm VRAM
#         gradient_accumulation_steps=2,   # mô phỏng batch lớn bằng cộng dồn gradient
#     )

#     # Tạo Trainer để huấn luyện mô hình
#     trainer = Trainer(
#         model=finetuned_model, # Mô hình cần huấn luyện (ở đây là model gốc chưa fine-tune)
#         args=training_args, # Tham số huấn luyện đã cấu hình
#         train_dataset=tokenized_datasets['train'], # Dữ liệu huấn luyện
#         eval_dataset=tokenized_datasets['validation'], # Dữ liệu đánh giá (validation)
#     )

#     trainer.train() # Bắt đầu huấn luyện mô hình
#     print("Training completed successfully!") # In thông báo hoàn thành huấn luyện


#    # Lưu checkpoint fine-tuned (thư mục mặc định ở working dir)
#     finetuned_model.save_pretrained("finetuned_model_2_epoch") 

import os, time, torch
from transformers import TrainingArguments, Trainer

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
finetuned_model.config.use_cache = False  #  tắt cache để tránh giữ tensor lớn
finetuned_model.gradient_checkpointing_enable()  #  chia nhỏ graph tính toán

if to_train:
    output_dir = f"./sql-training-{int(time.time())}"

    training_args = TrainingArguments(
        output_dir=output_dir, # Thư mục lưu kết quả huấn luyện
        learning_rate=5e-3,# Tốc độ học của mô hình
        num_train_epochs=1, # Số epoch huấn luyện
        per_device_train_batch_size=4,   # batch size nhỏ để phù hợp với VRAM (huấn luyện)
        per_device_eval_batch_size=4, # batch size nhỏ để phù hợp với VRAM (đánh giá)
        gradient_accumulation_steps=4,   # cộng dồn gradient để mô phỏng batch size lớn hơn
        weight_decay=0.01, # Hệ số weight decay để tránh overfitting (thường chọn 0.01)
        logging_steps=50, # Ghi log mỗi 50 bước
        eval_strategy="no", # tắt eval trong lúc train cho nhẹ
        save_strategy="epoch", # lưu model mỗi epoch
        fp16=True, # mix precision để tiết kiệm VRAM
        optim="adafactor", # tối ưu hóa Adafactor tiết kiệm bộ nhớ
        report_to="none", # tắt Weights & Biases để tránh lỗi trên Kaggle
    )

    trainer = Trainer(
        model=finetuned_model, # Mô hình cần huấn luyện
        args=training_args, # Tham số huấn luyện đã cấu hình
        data_collator=data_collator, # Hàm collate để padding theo batch
        train_dataset=tokenized_datasets["train"], # Dữ liệu huấn luyện
    )

    # Dọn sạch cache GPU trước khi train
    torch.cuda.empty_cache()
    trainer.train()

    print("✅ Training completed successfully!")
    finetuned_model.save_pretrained("finetuned_model_2_epoch")
    tokenizer.save_pretrained("finetuned_model_2_epoch")

Step,Training Loss
50,0.349100
100,0.084300
150,0.066800
200,0.052000
250,0.055800
300,0.050200
350,0.048700
400,0.046900
450,0.043600
500,0.039700


✅ Training completed successfully!


In [ ]:
# Dù đã fine-tune hay chưa, vẫn lưu model/tokenizer vào thư mục làm việc của Kaggle để tải về
finetuned_model.save_pretrained("/kaggle/working/sql_t5_finetuned") # Lưu model đã fine-tune vào thư mục làm việc của Kaggle
tokenizer.save_pretrained("/kaggle/working/sql_t5_finetuned") # Lưu tokenizer vào thư mục làm việc của Kaggle


('/kaggle/working/sql_t5_finetuned/tokenizer_config.json',
 '/kaggle/working/sql_t5_finetuned/special_tokens_map.json',
 '/kaggle/working/sql_t5_finetuned/spiece.model',
 '/kaggle/working/sql_t5_finetuned/added_tokens.json')

In [ ]:
# Hiển thị model đã fine-tune (hoặc model gốc nếu chưa fine-tune)
print("Model:", finetuned_model) 


Model: T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout

# ==============================
# 9) KIỂM THỬ MÔ HÌNH ĐÃ FINE-TUNE (ZERO-SHOT TRÊN MẪU)
# ==============================

In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Chuẩn bị model đã fine-tune để suy luận
finetuned_model.gradient_checkpointing_disable()  # tắt checkpointing khi inference
finetuned_model.config.use_cache = True           # bật lại cache cho generate nhanh
finetuned_model.eval()
finetuned_model.to(device)

# (tùy chọn với baseline)
original_model.gradient_checkpointing_disable() if hasattr(original_model, "gradient_checkpointing_disable") else None
try:
    original_model.config.use_cache = True
except:
    pass
original_model.eval()
original_model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
index = 0
question = dataset['test'][index]['question']
context  = dataset['test'][index]['context']
answer   = dataset['test'][index]['answer']

prompt = f"""Tables:
{context}

Question:
{question}

Answer:
"""

# Tokenize 'prompt' (chuỗi được tạo từ context + question) thành tensors PyTorch
# - Nguồn dữ liệu: `prompt` (lấy từ dataset['test'][index]['context'] và ['question'])
# - Input: prompt (str), tokenizer (T5Tokenizer), device ('cuda' hoặc 'cpu')
# - Output: inputs (dict) chứa 'input_ids' và 'attention_mask' dưới dạng torch.Tensor đã được chuyển lên device
inputs = tokenizer(prompt, return_tensors='pt').to(device)

with torch.no_grad():
    output_tokens = finetuned_model.generate( # dùng phương thức generate để sinh text
        inputs["input_ids"], # input_ids của prompt đã tokenized
        max_new_tokens=200, # Giới hạn tối đa 200 token mới sinh
    )

output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True) # Giải mã token thành text, bỏ qua token đặc biệt

print("-"*80)
print("INPUT PROMPT:\n", prompt)
print("-"*80)
print("BASELINE HUMAN ANSWER:\n", answer, "\n")
print("-"*80)
print("FINE-TUNED MODEL - ZERO SHOT:\n", output_text)


--------------------------------------------------------------------------------
INPUT PROMPT:
 Tables:
CREATE TABLE table_name_11 (date VARCHAR, away_team VARCHAR)

Question:
On what Date did the Away team essendon play?

Answer:

--------------------------------------------------------------------------------
BASELINE HUMAN ANSWER:
 SELECT date FROM table_name_11 WHERE away_team = "essendon" 

--------------------------------------------------------------------------------
FINE-TUNED MODEL - ZERO SHOT:
 SELECT date FROM table_name_11 WHERE away_team = "essendon"


# ==============================
# 10) ĐÁNH GIÁ SƠ BỘ BẰNG ROUGE (SUBSET 25 MẪU)
# ==============================
 Ghi chú: ROUGE đo độ “giống” chuỗi, không phải thước đo chuẩn cho SQL (Exec-Acc), nhưng dùng ở đây như chỉ báo tham khảo nhanh.


Các bước phía dưới sẽ thực hiện đánh giá định lượng mô hình đã fine-tune bằng metric ROUGE trên một tập mẫu nhỏ (25 mẫu) để so sánh chất lượng sinh SQL giữa model gốc và model đã huấn luyện.  
Sau đó, hướng dẫn cách sử dụng mô hình đã fine-tune để sinh truy vấn SQL mới (inference), và đóng gói mô hình thành file zip để tiện tải về hoặc triển khai vào ứng dụng thực tế (FastAPI, Flask, .NET...).

**Tóm tắt các bước:**
1. Đánh giá mô hình bằng ROUGE (so sánh với đáp án chuẩn).
2. Sinh truy vấn SQL mới từ mô hình đã fine-tune (inference).

In [ ]:
from transformers import GenerationConfig
import pandas as pd

questions = dataset['test'][0:25]['question']
contexts  = dataset['test'][0:25]['context']
human_baseline_answers = dataset['test'][0:25]['answer']

original_model_answers  = []
finetuned_model_answers = []

gen_cfg = GenerationConfig(max_new_tokens=300)

with torch.no_grad():
    for q, c in zip(questions, contexts):
        prompt = f"""Tables:
{c}

Question:
{q}

Answer:
"""
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

        # Baseline
        out_base = original_model.generate(input_ids=input_ids, generation_config=gen_cfg)
        original_model_answers.append(tokenizer.decode(out_base[0], skip_special_tokens=True))

        # Fine-tuned
        out_ft = finetuned_model.generate(input_ids=input_ids, generation_config=gen_cfg)
        finetuned_model_answers.append(tokenizer.decode(out_ft[0], skip_special_tokens=True))

df = pd.DataFrame(
    zip(human_baseline_answers, original_model_answers, finetuned_model_answers),
    columns=["human_baseline_answers", "original_model_answers", "finetuned_model_answers"]
)

display(df)  # nếu muốn xem bảng

,human_baseline_answers,original_model_answers,finetuned_model_answers
0,SELECT date FROM table_name_11 WHERE away_team...,Question,SELECT date FROM table_name_11 WHERE away_team...
1,SELECT institution FROM table_1974632_1 WHERE ...,"Question: state the institution in glenside, p...",SELECT institution FROM table_1974632_1 WHERE ...
2,SELECT home_team FROM table_name_4 WHERE away_...,True,SELECT home_team FROM table_name_4 WHERE away_...
3,SELECT date FROM table_name_49 WHERE home_team...,Question,SELECT date FROM table_name_49 WHERE home_team...
4,"SELECT ""Character"" FROM table_79388 WHERE ""Dur...",True,"SELECT ""Character"" FROM table_79388 WHERE ""Yea..."
5,SELECT clubs FROM table_name_59 WHERE position...,Question,SELECT clubs FROM table_name_59 WHERE position...
6,SELECT record FROM table_name_72 WHERE date = ...,Question,SELECT record FROM table_name_72 WHERE date = ...
7,SELECT DISTINCT flight.flight_id FROM airport_...,"CREATE TABLE flight_leg ( flight_id int, leg_n...",SELECT DISTINCT flight.flight_id FROM airport_...
8,SELECT season_joined_league FROM table_name_28...,Question,SELECT season_joined_league FROM table_name_28...
9,"SELECT loss FROM table_name_48 WHERE date = ""s...",True,"SELECT loss FROM table_name_48 WHERE date = ""s..."


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=3a9ae77b989177786d9963eaa176845032325aa644956df0df7a0c39f1389271
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


Compute ROUGE score for this subset of the data.

In [ ]:
rouge = evaluate.load('rouge') # Nạp metric ROUGE từ thư viện evaluate

original_model_results = rouge.compute( # Tính toán ROUGE cho model gốc
    predictions=original_model_answers, # Dự đoán từ model gốc
    references=human_baseline_answers[0:len(original_model_answers)], # Đáp án chuẩn tương ứng
    use_aggregator=True, # Sử dụng hàm tổng hợp kết quả
    use_stemmer=True, # Sử dụng stemming để cải thiện so khớp
)
print('ORIGINAL MODEL:')
print(original_model_results) # In kết quả ROUGE cho model gốc


finetuned_model_results = rouge.compute( # Tính toán ROUGE cho model đã fine-tune
    predictions=finetuned_model_answers, # Dự đoán từ model đã fine-tune
    references=human_baseline_answers[0:len(finetuned_model_answers)], # Đáp án chuẩn tương ứng
    use_aggregator=True,
    use_stemmer=True,
)
print('FINE-TUNED MODEL:')
print(finetuned_model_results) # In kết quả ROUGE cho model đã fine-tune

ORIGINAL MODEL:
{'rouge1': 0.031233998975934457, 'rouge2': 0.005, 'rougeL': 0.03151917519331407, 'rougeLsum': 0.03174603174603174}
FINE-TUNED MODEL:
{'rouge1': 0.9265132150912692, 'rouge2': 0.903794543074439, 'rougeL': 0.921148312345613, 'rougeLsum': 0.9178219960064249}


# ==============================
# 11) HÀM SUY LUẬN (INFERENCE) VỚI MÔ HÌNH ĐÃ FINE-TUNE
# ==============================

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer # Model và Tokenizer T5 (nạp lại để chắc chắn)
import torch

# Nạp mô hình đã fine-tune và tokenizer từ thư mục lưu trên Kaggle
model_path = "sql_t5_finetuned" # Thư mục ngay dưới /kaggle/working
model = T5ForConditionalGeneration.from_pretrained(model_path).to('cuda')
tokenizer = T5Tokenizer.from_pretrained(model_path)

# Hàm tiện ích để sinh truy vấn SQL từ context và question
def generate_sql(context, question):
    prompt = f"""Tables:
    {context}

    Question:
    {question}

    Answer:
    """
    
    inputs = tokenizer(prompt, return_tensors='pt').to('cuda')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=200,
        )[0],
        skip_special_tokens=True
    )

    return output

# Ví dụ inference với một mẫu từ tập test (phần tử đầu tiên)
index = 0
context = dataset['test'][index]['context']
question = dataset['test'][index]['question']

output = generate_sql(context, question)

print(f"Generated SQL Query:\n{output}")


Generated SQL Query:
SELECT date FROM table_name_11 WHERE away_team = "essendon"


# ==============================
# 12) ĐÓNG GÓI MÔ HÌNH ĐỂ TẢI XUỐNG
# ==============================

In [ ]:
# Cài thêm để chạy service nếu cần (không bắt buộc)
!pip install fastapi uvicorn transformers torch


In [ ]:
import shutil # Thư viện shutil để nén file zip

# Nén thư mục model đã fine-tune để tiện download từ Kaggle
shutil.make_archive(
    '/kaggle/working/sql_t5_finetuned',  # tên file zip đầu ra
    'zip',                               # định dạng nén
    '/kaggle/working/finetuned_model_2_epoch'  # thư mục model thật sự
)



'/kaggle/working/sql_t5_finetuned.zip'

**Nén toàn bộ dữ liệu quan trọng gồm model, dataset gốc, tokenized dataset, checkpoint training — để tải 1 lần duy nhất.**

In [ ]:
import shutil

# Đường dẫn thư mục làm việc
base_dir = "/kaggle/working"

# Đường dẫn file zip đầu ra
output_zip = f"{base_dir}/olms_sql_full_backup"

# Nén toàn bộ working directory (model + dataset + checkpoint)
shutil.make_archive(output_zip, 'zip', base_dir)

print("Đã nén toàn bộ dữ liệu thành công!")
print(f" File nén nằm tại: {output_zip}.zip")


Đã nén toàn bộ dữ liệu thành công!
 File nén nằm tại: /kaggle/working/olms_sql_full_backup.zip
